<a href="https://colab.research.google.com/github/cbeckler/final_project/blob/cb_etl/PM_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:14 http://security.ubuntu.c

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-01-06 20:58:04--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  4.83MB/s    in 0.2s    

2023-01-06 20:58:04 (4.83 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
# start spark session that can connect to postgre
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("final-project").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd

cancer_path = "/content/drive/MyDrive/Data/cancer_incidence.csv"

cancer_df = pd.read_csv(cancer_path)

cancer_df.head()

,index,County,FIPS,"Age-Adjusted Incidence Rate(Ê) - cases per 100,000",Lower 95% Confidence Interval,Upper 95% Confidence Interval,Average Annual Count,Recent Trend,Recent 5-Year Trend (ˆ) in Incidence Rates,Lower 95% Confidence Interval.1,Upper 95% Confidence Interval.1
0,0,"US (SEER+NPCR)(1,10)",0,62.4,62.3,62.6,214614,falling,-2.5,-3,-2
1,1,"Autauga County, Alabama(6,10)",1001,74.9,65.1,85.7,43,stable,0.5,-14.9,18.6
2,2,"Baldwin County, Alabama(6,10)",1003,66.9,62.4,71.7,170,stable,3,-10.2,18.3
3,3,"Barbour County, Alabama(6,10)",1005,74.6,61.8,89.4,25,stable,-6.4,-18.3,7.3
4,4,"Bibb County, Alabama(6,10)",1007,86.4,71,104.2,23,stable,-4.5,-31.4,32.9


In [6]:
# get rid of recent trend nulls (stored as *, _, __)

cancer_df = cancer_df.loc[(cancer_df['Recent Trend']=='rising')|(cancer_df['Recent Trend']=='falling')|(cancer_df['Recent Trend']=='stable')]

cancer_df = cancer_df[[' FIPS', 'Recent Trend']].copy()

cancer_df['Recent Trend'].value_counts()

stable     2429
falling     200
rising       43
Name: Recent Trend, dtype: int64

In [7]:
pm_path = "/content/drive/MyDrive/Data/Daily_PM2.5_Concentrations_All_County__2001-2016.csv"

pm_df = pd.read_csv(pm_path)

pm_df.head()

,year,date,statefips,countyfips,PM25_max_pred,PM25_med_pred,PM25_mean_pred,PM25_pop_pred
0,2001,01JAN2001,1,1,10.664367,10.264546,10.137631,10.188703
1,2001,01JAN2001,1,3,9.803209,8.739505,8.743748,8.811486
2,2001,01JAN2001,1,5,12.087599,11.809159,11.812775,11.802062
3,2001,01JAN2001,1,7,8.579425,8.435394,8.458118,8.448871
4,2001,01JAN2001,1,9,14.399446,13.577741,13.300528,13.231461


In [8]:
# filter pollution data for years matching cancer data
pm_df = pm_df.loc[(pm_df['year']<=2014)&(pm_df['PM25_med_pred']<=100)]

In [9]:
# check years
pm_df.year.unique()

array([2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011,
       2012, 2013, 2014])

In [10]:
# create state + county FIPS to match cancer data FIPS

pm_df['FIPS'] = pm_df['statefips'].astype(str) + pm_df['countyfips'].astype(str)

pm_df['FIPS'] = pm_df['FIPS'].astype(int)

pm_df.head()

,year,date,statefips,countyfips,PM25_max_pred,PM25_med_pred,PM25_mean_pred,PM25_pop_pred,FIPS
0,2001,01JAN2001,1,1,10.664367,10.264546,10.137631,10.188703,11
1,2001,01JAN2001,1,3,9.803209,8.739505,8.743748,8.811486,13
2,2001,01JAN2001,1,5,12.087599,11.809159,11.812775,11.802062,15
3,2001,01JAN2001,1,7,8.579425,8.435394,8.458118,8.448871,17
4,2001,01JAN2001,1,9,14.399446,13.577741,13.300528,13.231461,19


In [11]:
# aggregate pollution data by county

pm_agg = pm_df.groupby('FIPS').agg({'PM25_max_pred':'mean', 'PM25_med_pred':'mean', 'PM25_mean_pred':'mean'})

pm_agg.reset_index(inplace=True)

pm_agg.head()

,FIPS,PM25_max_pred,PM25_med_pred,PM25_mean_pred
0,11,12.239594,11.870589,11.787900
1,13,11.024418,9.822870,9.794272
2,15,11.688170,11.099414,11.084238
3,17,12.280141,11.700512,11.721958
4,19,13.498799,12.690525,12.741736


In [12]:
# get row count of initial df

len(cancer_df)

2672

In [13]:
# merge cancer and pollution data

final_df = pd.merge(cancer_df, pm_agg, how='inner', left_on=' FIPS', right_on='FIPS')

final_df.head()

,FIPS,Recent Trend,FIPS,PM25_max_pred,PM25_med_pred,PM25_mean_pred
0,1101,falling,1101,12.555164,12.013325,11.958884
1,1103,stable,1103,13.287901,12.684610,12.647047
2,1105,stable,1105,11.426605,11.072150,11.079387
3,1107,stable,1107,11.594499,11.146401,11.155309
4,1109,stable,1109,11.352271,10.939546,10.932512


In [14]:
# get row count of matches

len(final_df)

1048

In [15]:
# get count of label distribution

final_df['Recent Trend'].value_counts()

stable     961
falling     68
rising      19
Name: Recent Trend, dtype: int64

In [16]:
null_df = pd.merge(cancer_df, pm_agg, how='left', left_on=' FIPS', right_on='FIPS')

null_df.head()

,FIPS,Recent Trend,FIPS,PM25_max_pred,PM25_med_pred,PM25_mean_pred
0,0,falling,NaN,NaN,NaN,NaN
1,1001,stable,NaN,NaN,NaN,NaN
2,1003,stable,NaN,NaN,NaN,NaN
3,1005,stable,NaN,NaN,NaN,NaN
4,1007,stable,NaN,NaN,NaN,NaN


In [17]:
len(null_df)

2672

In [18]:
# get count of label distribution

null_df['Recent Trend'].value_counts()

stable     2429
falling     200
rising       43
Name: Recent Trend, dtype: int64

In [19]:
# aggregate pollution data by county and year

pm_year_agg = pm_df.groupby(['FIPS', 'year']).agg({'PM25_max_pred':'mean', 'PM25_med_pred':'mean', 'PM25_mean_pred':'mean'})

pm_year_agg.reset_index(inplace=True)

pm_year_agg.head()

,FIPS,year,PM25_max_pred,PM25_med_pred,PM25_mean_pred
0,11,2001,13.186807,12.834644,12.784293
1,11,2002,12.490668,12.162930,12.068693
2,11,2003,12.869481,12.485005,12.400802
3,11,2004,13.111945,12.716436,12.619426
4,11,2005,13.543276,13.164472,13.071456


In [20]:
# wide to long format

pm_year_agg = pd.melt(pm_year_agg, id_vars=['FIPS', 'year'], value_vars=['PM25_max_pred', 'PM25_med_pred', 'PM25_mean_pred'])

pm_year_agg['col_name'] = pm_year_agg['variable'] + '_' + pm_year_agg['year'].astype(str)

pm_year_agg = pm_year_agg.pivot(index='FIPS', columns='col_name', values='value')

pm_year_agg.reset_index(inplace=True)

pm_year_agg.head()

col_name,FIPS,PM25_max_pred_2001,PM25_max_pred_2002,PM25_max_pred_2003,PM25_max_pred_2004,PM25_max_pred_2005,PM25_max_pred_2006,PM25_max_pred_2007,PM25_max_pred_2008,PM25_max_pred_2009,...,PM25_med_pred_2005,PM25_med_pred_2006,PM25_med_pred_2007,PM25_med_pred_2008,PM25_med_pred_2009,PM25_med_pred_2010,PM25_med_pred_2011,PM25_med_pred_2012,PM25_med_pred_2013,PM25_med_pred_2014
0,11,13.186807,12.490668,12.869481,13.111945,13.543276,13.638252,14.542888,12.507304,10.279905,...,13.164472,13.224270,14.036194,12.022979,10.003785,10.788455,10.808470,10.438640,10.220977,11.282171
1,13,12.520190,11.269680,12.162325,12.081706,12.994364,11.942907,12.256714,10.514962,9.342412,...,11.503305,10.513930,10.622514,9.247900,8.429134,8.885592,8.865912,8.737817,8.437009,9.635918
2,15,12.364383,11.443971,11.531937,12.543327,12.787813,12.915748,14.095731,11.702072,10.480079,...,12.087744,12.229105,13.112620,11.074860,9.945611,10.689897,10.289959,9.641680,9.644445,10.925400
3,17,13.944184,12.781192,13.162910,13.082518,14.310805,14.060887,14.206462,12.319946,9.992209,...,13.484249,13.370442,13.508796,11.717692,9.575728,10.485379,10.607298,9.992089,9.996376,10.747303
4,19,16.032861,14.493547,14.722058,14.622386,15.809701,15.465637,16.052929,13.357497,10.911080,...,14.797208,14.530090,15.028200,12.598568,10.283973,11.372750,11.053671,10.687267,10.306375,10.994724


In [21]:
# merge year data

final_df = pd.merge(final_df, pm_year_agg, how='left', left_on='FIPS', right_on='FIPS')

final_df.head()

,FIPS,Recent Trend,FIPS,PM25_max_pred,PM25_med_pred,PM25_mean_pred,PM25_max_pred_2001,PM25_max_pred_2002,PM25_max_pred_2003,PM25_max_pred_2004,...,PM25_med_pred_2005,PM25_med_pred_2006,PM25_med_pred_2007,PM25_med_pred_2008,PM25_med_pred_2009,PM25_med_pred_2010,PM25_med_pred_2011,PM25_med_pred_2012,PM25_med_pred_2013,PM25_med_pred_2014
0,1101,falling,1101,12.555164,12.013325,11.958884,13.365598,12.650435,12.961279,13.413924,...,13.281816,13.213586,14.217639,11.965735,10.161459,11.003184,11.000639,10.832822,10.487398,11.657762
1,1103,stable,1103,13.287901,12.684610,12.647047,15.564124,14.211956,14.589012,14.135937,...,14.536614,14.079867,14.655598,12.577256,10.325899,11.515227,10.914625,11.124903,10.694759,11.419629
2,1105,stable,1105,11.426605,11.072150,11.079387,12.768430,11.553522,12.211885,12.179577,...,12.439048,12.648352,12.843697,11.171445,9.303155,10.035245,10.018046,9.397366,9.496719,10.343648
3,1107,stable,1107,11.594499,11.146401,11.155309,13.199240,11.938396,12.568184,12.204957,...,12.750975,12.486562,12.811823,11.247699,9.363321,10.145407,9.833054,9.473096,9.798761,10.097687
4,1109,stable,1109,11.352271,10.939546,10.932512,12.164619,11.319407,11.592101,12.247304,...,12.150266,12.124859,12.862628,10.830729,9.598606,10.293942,10.035339,9.412580,9.389697,10.631003


In [22]:
null_df = pd.merge(null_df, pm_year_agg, how='left', left_on='FIPS', right_on='FIPS')

null_df.head()

,FIPS,Recent Trend,FIPS,PM25_max_pred,PM25_med_pred,PM25_mean_pred,PM25_max_pred_2001,PM25_max_pred_2002,PM25_max_pred_2003,PM25_max_pred_2004,...,PM25_med_pred_2005,PM25_med_pred_2006,PM25_med_pred_2007,PM25_med_pred_2008,PM25_med_pred_2009,PM25_med_pred_2010,PM25_med_pred_2011,PM25_med_pred_2012,PM25_med_pred_2013,PM25_med_pred_2014
0,0,falling,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1001,stable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1003,stable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1005,stable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1007,stable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
